# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. It measures the stock performance of 500 large companies listed on stock exchanges in the US. It's widely regarded as one of the best indicators and is often used as a benchmark for the overall performance of the stock market. Investors and analysts use the S&P 500 to track the health of the U.S. economy and make investment decisions.

It is the market capitalization (or size) weighted. Larger the company, more weight in the index.
It's not necessarily to be exactly 500 companies because some companies have multiple share classes like Google-A, Google-C

## Library Imports

Firstly, let's import the open-source software libraries.

In [64]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing, is to import the constituents of the S&P 500.

I will be using the static version of the S&P 500 constituents available on. [link to download](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing)
   

In [65]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
491,YUM
492,ZBH
493,ZBRA
494,ZION


## Acquiring an API Token

Now it's time to import the IEX Cloud API token. This is the data provider through which I will extract the required data.

##### NOTE : - API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to local Git repository. Use sandbox API token to randomly-generated the data. This was free earlier but now, can be used under the free trial option.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [66]:
IEX_CLOUD_API_TOKEN = "pk_9cb6a39b4a834a2da6a683c795c0346f"

## Making my First API Call

Now it's time to structure the API calls to IEX cloud. 

Extract the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [67]:
name='AAPL'
api_url = f'https://api.iex.cloud/v1/stock/{name}/quote/?token={IEX_CLOUD_API_TOKEN}'
data=requests.get(api_url).json()
data

{'avgTotalVolume': 58657720,
 'calculationPrice': 'tops',
 'change': 2.33,
 'changePercent': 0.01261,
 'close': None,
 'closeSource': 'official',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': None,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 187.14,
 'iexAskSize': 200,
 'iexBidPrice': 187.13,
 'iexBidSize': 100,
 'iexClose': 187.13,
 'iexCloseTime': 1699989297850,
 'iexLastUpdated': 1699989297850,
 'iexMarketPercent': 0.0268030308912569,
 'iexOpen': 187.69,
 'iexOpenTime': 1699972200222,
 'iexRealtimePrice': 187.13,
 'iexRealtimeSize': 1,
 'iexVolume': 920982,
 'lastTradeTime': 1699989297850,
 'latestPrice': 187.13,
 'latestSource': 'IEX real time price',
 'latestTime': '2:14:57 PM',
 'latestUpdate': 1699989297850,
 'latestVolume': None,
 'low': None,
 'lowSource': None,
 'lowTime': Non

## Parsing the API Call

The API call generated earlier in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. Parse it first.

In [68]:
price=data['latestPrice']
market_cap=data['marketCap']

## Adding the Stocks Data to a Pandas DataFrame

The next thing is to add the stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data. It stands for panel data and makes it easy to work with tabular data in python for easy data analysis.

Pandas data frame refers to the data structure of pandas that stores tabular data (2-D).
Pandas series refers to the data structure of pandas that stores data in 1-D.

In [69]:
my_columns = [ 'Ticker', 'Stock Price', 'Market Capitalization', 'Number of shares to Buy']
new_dataFrame = pd.DataFrame(columns = my_columns)
new_dataFrame

,Ticker,Stock Price,Market Capitalization,Number of shares to Buy


In [70]:
new_dataFrame = pd.DataFrame(columns = my_columns)
new_columns=[name,price,market_cap,'N/A']
new_dataFrame.loc[len(new_dataFrame)] = new_columns
new_dataFrame

,Ticker,Stock Price,Market Capitalization,Number of shares to Buy
0,AAPL,187.13,2910386481760,N/A


## Looping Through The Tickers in List of Stocks

Using the same logic outlined above, pull data for all S&P 500 stocks and store them in the DataFrame using a `for` loop.

In [71]:
new_dataFrame = pd.DataFrame(columns = my_columns)
for name in stocks['Ticker'][:4] : 
    api_url = f'https://api.iex.cloud/v1/stock/{name}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(api_url).json()
    new_columns=[name,data['latestPrice'],data['marketCap'],'N/A']
    new_dataFrame.loc[len(new_dataFrame)] = new_columns
new_dataFrame

,Ticker,Stock Price,Market Capitalization,Number of shares to Buy
0,A,111.370,32585444594,N/A
1,AAL,12.265,8015674846,N/A
2,AAP,58.630,3486741402,N/A
3,AAPL,187.130,2910386481760,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of the code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls from 500 to 5 - huge improvement! Now, I will split my list of stocks into groups of 100 and then make a batch API call for each group.

In [72]:
def partition(lst, n):
#Yield successive n-sized partition from list

    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [73]:
#creating a list of lists of each list having max size 100
#bcz batch API calls 100 tickers per request

symbol_groups = list(partition(stocks['Ticker'], 100))
symbol_strings = [] # create empty string
final_columns = [ 'Ticker', 'Stock Price', 'Market Capitalization', 'Number of shares to Buy']
final_dataFrame = pd.DataFrame(columns = final_columns)

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

for i in symbol_strings:
        batch_api_call_url = f'https://api.iex.cloud/v1/stock/market/batch/?types=quote&symbols={i}&token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(batch_api_call_url).json()
        
        for name in i.split(','):
            new_columns=[name,data[name]['quote']['latestPrice'],data[name]['quote']['marketCap'],'N/A']
            final_dataFrame.loc[len(final_dataFrame)] = new_columns
final_dataFrame

,Ticker,Stock Price,Market Capitalization,Number of shares to Buy
0,A,111.370,32585444594,N/A
1,AAL,12.265,8015674846,N/A
2,AAP,58.630,3486741402,N/A
3,AAPL,187.130,2910386481760,N/A
4,ABBV,138.560,244632865054,N/A
...,...,...,...,...
491,YUM,127.380,35705660936,N/A
492,ZBH,107.830,22534390067,N/A
493,ZBRA,214.725,11028190969,N/A
494,ZION,35.190,5213356237,N/A


## Calculating the Number of Shares to Buy

It's time to calculate the number of shares of each stock to buy.

In [74]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \nTry again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:1000000


In [75]:
position_size = float(portfolio_size) / len(final_dataFrame.index)
for i in range(0, len(final_dataFrame['Ticker'])-1):
    final_dataFrame.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataFrame.loc[i, 'Stock Price'])
final_dataFrame

,Ticker,Stock Price,Market Capitalization,Number of shares to Buy,Number Of Shares to Buy
0,A,111.370,32585444594,N/A,18.0
1,AAL,12.265,8015674846,N/A,164.0
2,AAP,58.630,3486741402,N/A,34.0
3,AAPL,187.130,2910386481760,N/A,10.0
4,ABBV,138.560,244632865054,N/A,14.0
...,...,...,...,...,...
491,YUM,127.380,35705660936,N/A,15.0
492,ZBH,107.830,22534390067,N/A,18.0
493,ZBRA,214.725,11028190969,N/A,9.0
494,ZION,35.190,5213356237,N/A,57.0


## Formatting Our Excel Output

Using the XlsxWriter library for Python to create nicely-formatted Excel files and save those excel documents from python script.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users.

### Initializing our XlsxWriter Object

In [85]:
writer = pd.ExcelWriter('recommended_trades_s&p500.xlsx', engine='xlsxwriter')
final_dataFrame.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. The four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [86]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of `.xlsx` File

Use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [87]:
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)
# writer.close()

In [88]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format],
                    'E': ['Number of Shares to Buy', integer_format]
                 }

for i in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{i}:{i}', 20, column_formats[i][1])
    writer.sheets['Recommended Trades'].write(f'{i}1', column_formats[i][0], column_formats[i][1])

## Finally, saving the Excel Output !!

In [89]:
writer.close()